In [ ]:
DIR = "timing/";

|-----------|




### Basic Framework

In [ ]:
PATH_TO_GAPSE = "../";

In [ ]:
include(PATH_TO_GAPSE * "src/GaPSE.jl");

In [ ]:
using DelimitedFiles, Plots, Dierckx, SpecialFunctions
using Printf, LaTeXStrings, Turing, StatsPlots, GridInterpolations

In [ ]:
z_min, z_max, θ_max = 0.05, 0.20, π / 2.0
#z_min, z_max, θ_max = 1.0, 1.5, π / 2.0

WFI_opts = Dict(
    :ss_start => 0.0, :ss_stop => 0.0, 
    :ss_step => 50, :llim => 0.0, :rlim => Inf, 
    :rtol => 5e-2, :atol => 0.0, :N => 1000, #:pr => true,
)

FILE_F_MAP = PATH_TO_GAPSE * "data/F_REFERENCE_pi2.txt"
FILE_PS = PATH_TO_GAPSE * "test/datatest/file_pk.txt"
FILE_BACKGROUND = PATH_TO_GAPSE * "test/datatest/WideA_ZA_background.dat"

params = GaPSE.CosmoParams(z_min, z_max, θ_max;
    Ω_b = 0.0489, Ω_cdm = 0.251020, h_0 = 0.70, s_lim=1e-2,
    s_b = 0.0, 𝑓_evo = 0.0, b = 1.0,
    IPS_opts = Dict(
        :fit_left_min => 1e-6, :fit_left_max => 3e-6,
        :fit_right_min => 1e1, :fit_right_max => 2e1),
    IPSTools_opts = Dict(
        :N => 1024, :fit_min => 0.05, :fit_max => 0.5, 
        :con => true, :k_min => 1e-8, :k_max => 10.0),
    WFI_opts = WFI_opts,
);

In [ ]:
#=
new_calc_μs = union([μ for μ in -1.0:0.01:(-0.9)], 
        [μ for μ in -0.9:0.05:0.9], 
        [μ for μ in 0.9:0.01:1.0]);
FILE_IF_MAP = PATH_TO_GAPSE*"data/IntF_pi2_z115.txt"

GaPSE.print_map_IntegratedF(
    FILE_F_MAP, FILE_IF_MAP, 
    z_min, z_max, new_calc_μs, FILE_BACKGROUND;
    trap = true, WFI_opts...
)
=#

In [ ]:
#FILE_IF_MAP = PATH_TO_GAPSE*"data/IntF_pi2_z115.txt"
FILE_IF_MAP = PATH_TO_GAPSE*"data/IntegrF_REFERENCE_pi2_z005020.txt"
cosmo = GaPSE.Cosmology(params, FILE_BACKGROUND, FILE_PS, FILE_F_MAP, FILE_IF_MAP);

GaPSE.parameters_used(stdout, cosmo)

### noF L=0

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L0_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L0_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L0_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

### noF L=2

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L2_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L2_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L2_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

### noF L=4

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L4_noF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L4_noF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L4_noF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = false, enhancer = 1e8, L=4,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

### with F L=0

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L0_withF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L0_withF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L0_withF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = true, enhancer = 1e8, L=0,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);

### with F L=2

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Newtonian_L2_withF.txt", 
    "auto_newton",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true);

In [ ]:
GaPSE.print_map_ξ_GNC_multipole(
    cosmo, DIR*"map_xis_Lensing_L2_withF.txt", 
    "auto_lensing",
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, N_χs_2 = 60);

In [ ]:
GaPSE.print_map_sum_ξ_GNC_multipole(
    cosmo, DIR*"map_sum_xis_GNC_L2_withF.txt", 
    10 .^ range(0, log10(2*cosmo.s_max), length = 500);
    single = true, use_windows = true, enhancer = 1e8, L=2,
    N_μs = 200, μ_atol = 0.0, μ_rtol = 1e-2, trap = true, 
    N_χs = 100, N_χs_2 = 60);